In [1]:
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import models
import training
from torchvision.transforms import v2
import numpy as np
import matplotlib.pyplot as plts
from os.path import join
import torchvision
import torch.nn.functional as F
import h5py
import matplotlib.pyplot as plt
from torch.utils.data import random_split

device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def format_data(x):
    mean = [x[:, n, :, :].mean() for n in range(x.shape[1])]
    std = [x[:, n, :, :].std() for n in range(x.shape[1])]

    X = v2.Compose([
            torch.from_numpy,
            v2.Normalize(mean=mean, std=std),
            v2.Resize((64, 64)),
        ])(x).to(device)
    return X

Using cuda device


In [19]:
orders = range(1,2)

for order in orders:
    with h5py.File('../../Data/Training/intense_mixed.h5', 'r') as f:
        images = format_data(f[f'images_order{order}'][:])
        labels = torch.from_numpy(f[f'labels_order{order}'][:]).to(device)

    dset = TensorDataset(images, labels)

    train_size = int(0.85 * len(dset))
    test_size = len(dset) - train_size

    train_dataset, test_dataset = random_split(dset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256)

    L = images.shape[2]
    loss_fn = torch.nn.MSELoss()
    n_channels = 2
    n_classes = (order+1)**2-1

    model = models.ConvNet(L,L,n_channels, n_classes,[24,40,35],5,nn.ELU,[120,80,40]).to(device)

    optimizer = torch.optim.Adam(model.parameters(), amsgrad=True)

    save_path = f"../../Results/MachineLearningModels/Intense/Mixed_Order{order}_v2"

    writer = SummaryWriter(save_path)
    early_stopping = training.EarlyStopping(patience=50,save_path=save_path)
    for t in range(200):
        epoch = t+1
        print(f"-------------------------------\nEpoch {epoch}")
        training.train(model, train_loader, loss_fn, optimizer, device)
        val_loss = training.test(model, test_loader, loss_fn, device, epoch, writer, verbose=True)
        early_stopping(val_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break
    print("Done!")
    writer.close()
        

-------------------------------
Epoch 1
Training loss: 0.00270086
-------------------------------
Epoch 2
Training loss: 0.00294001
-------------------------------
Epoch 3
Training loss: 0.00250214
-------------------------------
Epoch 4
Training loss: 0.00321586
-------------------------------
Epoch 5
Training loss: 0.00231482
-------------------------------
Epoch 6
Training loss: 0.00213476
-------------------------------
Epoch 7
Training loss: 0.00246051
-------------------------------
Epoch 8
Training loss: 0.00209666
-------------------------------
Epoch 9
Training loss: 0.00201559
-------------------------------
Epoch 10


In [16]:
from scipy.linalg import sqrtm
def A(dim, j):
    assert dim > j+1, "For a matrix of dimension dim, j must be smaller than dim-1."
    A = np.zeros(dim, dtype=np.float32)
    for i in range(j+1):
        A[i] = 1
    A[j+1] = -j - 1
    A = A / np.linalg.norm(A)
    return np.diag(A)

def B(dim, j, k):
    B = np.zeros((dim, dim), dtype=np.float32)
    B[j, k] = 1
    B[k, j] = 1
    B = B / np.linalg.norm(B)
    return B

def C(dim, j, k):
    C = np.zeros((dim, dim), dtype=np.complex64)
    C[j, k] = -1j
    C[k, j] = 1j
    C = C / np.linalg.norm(C)
    return C

def get_basis(dim):
    As = np.stack([A(dim, j) for j in range(0, dim-1)])
    Bs = np.empty((dim * (dim - 1) // 2, dim, dim), dtype=np.float32)
    Cs = np.empty((dim * (dim - 1) // 2, dim, dim), dtype=np.complex64)
    counter = 0
    for j in range(1, dim):
        for k in range(0, j):
            Bs[counter, :, :] = B(dim, j, k)
            Cs[counter, :, :] = C(dim, j, k)
            counter+=1
    return np.concatenate((As, Bs, Cs), axis=0)

def real_representation(rhos):
    assert rhos.shape[1] == rhos.shape[2], "The density matrices must be square."
    dim = rhos.shape[1]
    xs = np.empty((rhos.shape[0],dim**2 - 1), dtype=np.float32)
    basis = get_basis(dim)
    for rho, x in zip(rhos,xs):
        for i, b in enumerate(basis):
            x[i] = np.real(np.trace(rho @ b))
    return xs

def complex_representation(xs):
    dim = int(np.sqrt(xs.shape[1] + 1))
    basis = get_basis(dim)
    
    return np.stack([np.eye(dim) / dim + sum(c * b for c, b in zip(x, basis)) for x in xs])


def fidelity(rhos1, rhos2):
    fids = np.empty(rhos1.shape[0], dtype=np.float32)
    for i, (rho1, rho2) in enumerate(zip(rhos1, rhos2)):
        sqrt_rho1 = sqrtm(rho1)
        fids[i] = np.real(np.trace(sqrtm(sqrt_rho1 @ rho2 @ sqrt_rho1)))**2
    return fids

In [17]:
orders = range(1,2)

metric = fidelity
metrics = np.empty(len(orders))
metrics_std = np.empty(len(orders))


for order in orders:
    with h5py.File('../../Data/Processed/mixed_intense.h5') as f:
        images_exp = format_data(f[f'images_order{order}'][:])
        labels_exp = f[f'labels_order{order}'][:]
    

    X_exp = format_data(np.float32(x_exp))
    model = torch.load(f"../../Results/MachineLearningModels/Intense/Mixed_Order{order}/checkpoint.pt")

    with torch.no_grad():
        labels_pred = model(X_exp).cpu().numpy()
        with h5py.File("../../Results/Intense/machine_learning.h5", 'w-') as out:
            out.create_dataset(f'pred_labels_order{order}', data=labels_pred)
            out.create_dataset(f'labels_order{order}', data=labels_exp)
labels_pred

(array([0.97833139]), array([0.02050678]))

In [ ]:
with h5py.File('../../Results/Intense/machine_learning.h5','w') as f:
    f.create_dataset("fids", data=metrics)
    f.create_dataset("fids_std", data=metrics_std)